#### ライブラリの読み込み

In [1]:
import pickle
import numpy as np

#### データの読み込み

In [2]:
with open("../dataset/exp0153_resnet18_focal_mixup_pseudo0.5_thr0.5.pkl", "rb") as f:
    pseudo_labels = pickle.load(f)

#### サンプルラベル
気をつけてほしいのは末尾の `_posi`とか`_nega`は実際のpositive, negativeには関係ないこと  
ラベルがtp_trainからきたか、fp_trainからきたかの区別  
なのでかぶっているものもあるので注意(使うときによしなに統合する)

In [3]:
sample_posi = "0099c367b_posi"

#### sample_posiを確認
8 frame x 24 species_id

In [4]:
labels = pseudo_labels[sample_posi]
print(labels.shape)

(8, 24)


#### 8フレームについて
[Theo Vielさんのデータ](https://www.kaggle.com/c/rfcx-species-audio-detection/discussion/198048)を使っている  
画像データは3751x128ピクセルだが、これを512の窓で分割する  
境目にデータがくることを想定して49ずつかぶせる  

つまり各フレームは実際のサイズと以下のような対応となる

|フレーム|ピクセル|時間(s)|
|--|--|--|
|0|0〜512|0〜8|
|1|463〜975|7〜15|
|2|926〜1438|14〜23|
|3|1389〜1901|22〜30|
|4|1852〜2364|29〜37|
|5|2315〜2827|37〜45|
|6|2778〜3290|44〜52|
|7|3241〜3753|51〜60|

#### ラベルの確認
- 5-foldのモデルが3個以上、0.5以上と予測した場合にpositiveラベル`1`
- 5-foldのモデルが3個以上、0.01以下と予測した場合にnegativeラベル`-1`
- 上記以外は0

In [5]:
species_id_17 = labels[:, 17]
np.where(species_id_17==1)

(array([0, 3, 4, 5, 6, 7]),)

0〜8秒、22〜60秒の区間にspecies_idが17の声が出現

In [6]:
species_id_13 = labels[:, 13]
np.where(species_id_13==-1)

(array([3]),)

22〜30秒の区間にspecies_idが13の声は無い